In [63]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
import pandas as pd

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

data = pd.read_csv(r"./Tweets.csv")
data.head(5)

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [64]:
X_train, X_test, y_train, y_test = train_test_split(data["text"], data["sentiment"], test_size=0.2, stratify=data["sentiment"])

len(X_train), len(X_test)

(21984, 5497)

In [65]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    if not isinstance(text, str) or text.strip() == "":
        return ""
    tokens = word_tokenize(text)
    tokens = [w for w in tokens if w.lower() not in stop_words]
    tokens = [lemmatizer.lemmatize(w.lower()) for w in tokens]
    return tokens

X_train = X_train.fillna("").apply(preprocess)
X_test = X_test.fillna("").apply(preprocess)
X_train[:5]

15082    [http, :, //tinyurl.com/c33ffv, thats, im, mak...
423      [_dec0de, finally, 1, hour, history, ..., 3.00...
19811                          [*, *, *, *, hate, goodbye]
11855    [given, way, kunal, khemu, etc, ., starrers, t...
61                                             [haha, yes]
Name: text, dtype: object

In [66]:
all_tokens = [token for tokens in X_train for token in tokens]

vocab = {token: idx+2 for idx, (token, _) in enumerate(Counter(all_tokens).most_common())}
vocab["<PAD>"] = 0
vocab["<UNK>"] = 1
vocab_size = len(vocab)

def encode(text):
    return [vocab.get(token, vocab["<UNK>"]) for token in text]

X_train = [encode(text) for text in X_train]
X_test= [encode(text) for text in X_test]

def pad_sequences(seqs, pad_value=0):
    return pad_sequence([torch.tensor(s) for s in seqs], batch_first=True, padding_value=pad_value)

X_train = pad_sequences(X_train)
X_test = pad_sequences(X_test)


X_test[1]

tensor([    1,  1620, 19886,    44,   846,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0])

In [67]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

In [68]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

class Network(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, bidirectional=True):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True, bidirectional=bidirectional)
        self.fc = nn.Linear(hidden_dim * (2 if bidirectional else 1), output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        output, hidden = self.rnn(embedded)
        if self.rnn.bidirectional:
            hidden = torch.cat((hidden[-2], hidden[-1]), dim=1)
        else:
            hidden = hidden[-1]
        return self.fc(hidden)

Using mps device


In [69]:
N = 100
train_dataloader = DataLoader(train_dataset, batch_size=N)
test_dataloader = DataLoader(test_dataset, batch_size=N)

def train(model, dataloader, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()

    for i, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i % 100 == 0:
            loss, current = loss.item(), i * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(model, dataloader, loss_fn):
    size = len(dataloader.dataset)
    num_N = len(dataloader)
    
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss = loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_N
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [70]:
embedding_dim = 100
hidden_dim = 1028
output_dim = len(le.classes_)
model = Network(vocab_size, embedding_dim, hidden_dim, output_dim, bidirectional=True).to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [72]:
epochs = 10

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-3)

for i in range(epochs):
    print(f"Epoch {i+1}\n-------------------------------")
    train(model, train_dataloader, loss_fn, optimizer)
    test(model, test_dataloader, loss_fn)

print("Done!")

Epoch 1
-------------------------------
loss: 0.534497  [    0/21984]
loss: 0.481728  [10000/21984]
loss: 0.660533  [20000/21984]
Test Error: 
 Accuracy: 62.4%, Avg loss: 0.015863 

Epoch 2
-------------------------------
loss: 0.466821  [    0/21984]
loss: 0.422031  [10000/21984]
loss: 0.409683  [20000/21984]
Test Error: 
 Accuracy: 64.6%, Avg loss: 0.015984 

Epoch 3
-------------------------------
loss: 0.470519  [    0/21984]
loss: 0.408879  [10000/21984]
loss: 0.395124  [20000/21984]
Test Error: 
 Accuracy: 62.0%, Avg loss: 0.015906 

Epoch 4
-------------------------------
loss: 0.464852  [    0/21984]
loss: 0.407001  [10000/21984]
loss: 0.394958  [20000/21984]
Test Error: 
 Accuracy: 61.7%, Avg loss: 0.016011 

Epoch 5
-------------------------------
loss: 0.456309  [    0/21984]
loss: 0.434199  [10000/21984]
loss: 0.385118  [20000/21984]
Test Error: 
 Accuracy: 64.6%, Avg loss: 0.015995 

Epoch 6
-------------------------------
loss: 0.455003  [    0/21984]
loss: 0.395941  [100

In [82]:
text = input("Text: ")
text = preprocess(text)
text = encode(text)
text = torch.tensor(text).to(device)
text = text.unsqueeze(0)

pred = model(text)
pred = le.inverse_transform([pred.argmax(1).item()])
pred[0]

'positive'